In [1]:
import pandas as pd
import numpy as np
import math
from bridge_specs import box
from Cross import calc_centroid as cg
from irc6_2007 import *



In [2]:
def loader(unit_wt, length, width, height, type):
    if type == 'r':
        load = unit_wt*length*width*height
    elif type == 't':
        load = unit_wt*length*width*height*0.5
    
    return load
def z(base, clr, dim, x, type):
    """base = width of base along with taper
       clr = overhang part from where centroid is calculated
        dim = width of object whose lever arm is required
        type = shape of object
    """
    y = []
    if type == 'r':
        y.append(cg('rectangle',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't1':
        y.append(cg('triangle_2',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't2':
        y.append(cg('triangle_1',[dim,0],[x,0]))
        return (-(clr+base/2)+y[0][0])

Abutment details

In [3]:
bearing_ht = 0.3 #assumed including pedestal
abut = {
    'total_ht' : 7.0,
    'wc' : 0.075,
    'backw_ht' : box['pillar_hei']+bearing_ht+0.075, #including wearing cource
    'cap':0.5,
    'stem_ht': 7-box['pillar_hei']+bearing_ht+0.075-0.5,
    'bw_width':0.3,
    'edge_to_bearing_c':0.6,
    'bw_to_bearing_c': 0.6,
    'cap_width':0.3+0.6+0.6,
    'stem_width':1.25,
    'be_taperwidth':0.10,
    'width_of_abut':8.4,
    'ttl_stembase_width':1.25+0.10,
    'approach_slab_dep':0.3,
    'approach_slab_len':3.5,
    'haunch':{'stem_haunch':[0.2,0.25],'approach_slab_haunch_rec':[0.25,0.25], 'approach_slab_haunch_tri':[0.25,0.25]} #list of width and height.
}
print(abut['backw_ht'])

3.575


Inputs: Live Loads, Dead loads and Pedestrian loads

In [4]:
#dummy loads
dl = 3000 #dead loads
ll = 1000 #live loads including impact
pdl = 200 #pedestrian loads
sdl=250 #Surface Loads

Input: Material properties

In [5]:
fck = 30 #grade of concrete = M30
fy = 500 #grade of steel = Fe500
unitwt_conc = 25 #unit weight of concrete = 25KN/m^3
unitwt_bfill = 20 #unit weight of backfill = 20KN/m^3

Self weight loads of abutment

In [6]:
# comp = ['stem_rect','stem_tri','cap','backwall','stem_haunch','approach_slab_haunch']
loads = [loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['stem_width'],'r'),
         loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['be_taperwidth'],'t'),
         loader(unitwt_conc,abut['cap'],abut['width_of_abut'],abut['cap_width'],'r'),
         loader(unitwt_conc,abut['backw_ht'],abut['width_of_abut'],abut['bw_width'],'r'),
         loader(unitwt_conc,abut['haunch']['stem_haunch'][0],abut['width_of_abut'],abut['haunch']['stem_haunch'][1],'r'),
         loader(unitwt_conc,abut['haunch']['approach_slab_haunch_rec'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_rec'][1],'r'),
          loader(unitwt_conc,abut['haunch']['approach_slab_haunch_tri'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_tri'][1],'t')]
lever_arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['stem_width'],abut['haunch']['stem_haunch'][0],'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['be_taperwidth'],abut['haunch']['stem_haunch'][0]+abut['stem_width'],'t1'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['cap_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['bw_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][0],0,'t2'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_rec'][0],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_tri'][0],0,'t2'),5)]
moments = []
for index,i in enumerate(loads):
    moments.append(loads[index]*lever_arm[index])
A1 = pd.DataFrame({"Load":loads,"Lever arm":lever_arm,"Moment":moments},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
A1.loc[len(A1.index)] = [A1['Load'].sum(),None,A1['Moment'].sum()]
display(A1.rename(index={7:'Total'}))

,Load,Lever arm,Moment
Stem(rectangle),964.6875,-0.05000,-48.234375
Stem(triangle),38.5875,0.60833,23.473934
Cap,157.5000,-0.12500,-19.687500
Backwall,225.2250,-0.72500,-163.288125
Stem haunch,10.5000,-0.74167,-7.787535
Approach slab haunch(rec),13.1250,-1.00000,-13.125000
Approach slab haunch(tri),6.5625,-1.04167,-6.835959
Total,1416.1875,NaN,-235.484560


Loads from superstructure

In [7]:
load = [dl,ll+pdl,sdl]
arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5)]
moment = []
for index,i in enumerate(load):
    moment.append(load[index]*arm[index])
A2 = pd.DataFrame({'Load':load, 'Lever arm':arm,'Moment':moment}, index = ['Dead load','Live load','Surface load'])
display(A2)

,Load,Lever arm,Moment
Dead load,3000,0.025,75.00
Live load,1200,0.025,30.00
Surface load,250,0.025,6.25


Earth Pressure as per clause 214 of IRC:6

In [8]:
phi = 33 #Angle of internal friction of soil
alpha = 0 #Angle which earth face of the wal makes with the vertical of the wall
beta = 0 #Slope of earth fill
delta = 2/3*phi if 2/3*phi<22.5 else 22.5 #Angle of friction between earth and earth fill
h = abut['total_ht']-abut['approach_slab_dep'] #Height of earthfill


In [9]:
ep = unitwt_bfill*h*ka(alpha, beta, delta, phi) #Earth pressure in KN/m^2.
force = 0.5*ep*abut['width_of_abut']*h #Force due to earth pressure in KN.
forc = [force*math.cos(math.radians(delta)),force*math.cos(math.radians(delta)),force*math.sin(math.radians(delta))] #Horizontal and vertical components.
larm = [0.42*h, 0.33*h, -(abut['stem_width']+abut['be_taperwidth'])/2 ] #0.42 times height of wall above base for dry soil and 0.33 for wet.
momen = []
for index,i in enumerate(forc):
    momen.append(forc[index]*larm[index])
A3 = pd.DataFrame({'Force(KN)':forc, 'Lever arm(m)':larm,'Moment(KNm)':momen}, index = ['Earth pressure Hz(dry)','Earth pressure Hz(wet)','Earth pressure Ver'])
display(A3)

,Force(KN),Lever arm(m),Moment(KNm)
Earth pressure Hz(dry),924.601823,2.814,2601.829531
Earth pressure Hz(wet),924.601823,2.211,2044.294631
Earth pressure Ver,373.563385,-0.675,-252.155285


Braking Forces

In [10]:
wt_70r = 1000 #Weight of 70R vehicle in KN.
classa_wt = 554 #Weight of Class A in KN.
pt = 1.2 #Point of application of load above the deck in m.
factor = 0.2 #Lane fact as per clause 211.2 of IRC 6

In [11]:
F = [1*wt_70r*factor, 2*classa_wt*factor]
la = pt+abut['total_ht']
m = max(F)*la
A4 = pd.DataFrame({'Force(KN)':max(F), 'Lever arm(m)':la,'Moment(KNm)':m}, index = ['Braking load'])
display(A4)

,Force(KN),Lever arm(m),Moment(KNm)
Braking load,221.6,8.2,1817.12


Temperature Forces

In [12]:
temp_var = 30 #Temperature variation in degree celsius
alp = 0.000012 #Coefficient of thermal expansion in m/deg. c/m
l = box['span'] #span of bridge
shr_str = 0.0002 #Strain due to shrinkage.
th_el = temp_var*alp*l #Thermal elongation in m.
shr_el = l*shr_str #Shrinkage elongation in m.
tot = (th_el+shr_el)/2 #Total elongation in m.
shr_rt = 1000 #Shear rating of elastomer bearing in KN/m/m^2
arb = 0.235644 #Area of bearing in m^2.
htb = 0.0340 #Height of bearing in m.
no = 3 #Number of bearings.
f = 1.1*tot*(shr_rt*arb)/htb

In [13]:
frc = 3*f
La = abut['stem_ht']+abut['cap']+bearing_ht
A5 = pd.DataFrame({'Force(KN)':frc, 'Lever arm(m)':La,'Moment(KNm)': (frc*La)}, index = ['Temperature force'])
display(A5)


,Force(KN),Lever arm(m),Moment(KNm)
Temperature force,320.198612,4.475,1432.888788


Live Load Surcharge as per clause 214.1.1.3 of IRC 6

In [14]:
hl = 1.2 
Pr = ka(alpha, beta, delta, phi)*hl*unitwt_bfill #Horizontal pressure 
A6 = pd.DataFrame({'Force(KN)':(Pr*abut['width_of_abut']*(abut['total_ht']-abut['approach_slab_dep'])), 'Lever arm(m)':abut['total_ht']/3,'Moment(KNm)':((Pr*abut['width_of_abut']*(abut['total_ht']-abut['approach_slab_dep']))*abut['total_ht']/3)}, index = ['Force due to LL surcharge'])
display(A6)


,Force(KN),Lever arm(m),Moment(KNm)
Force due to LL surcharge,357.211411,2.333333,833.493293


Seismic loads as per clause 218 of IRC 6

In [15]:
"""Inputs"""
zf = zone_factor(5) 
d = 1000 #Seismic importance factor as per table 19 of IRC 6
i = 1.2
f = 250 # Average response acceleration coefficient (see clause 218.5.1)
rrf = 3.0 #Response reduction factor as per table 20 of IRC 6 
ahh = ah(zf,sag(vib_period(d,f),'I'),rrf,1.2)
av = ahh*2/3
lam = max(math.degrees(math.atan(ahh/(1+av))),math.degrees(math.atan(ahh/(1-av))))
print(lam)

11.560130794217766


Due to superstructure

In [16]:
H_force = []
for index, i in enumerate(loads):
    H_force.append(round(loads[index]*ahh,5))
ht = [cg('rectangle', [abut['stem_width'],abut['stem_ht']], [0,0])[1], 
      cg('triangle_2', [abut['be_taperwidth'],abut['stem_ht']], [0,0])[1],
      cg('rectangle', [abut['cap_width'],abut['cap']], [0,abut['stem_ht']])[1],
      cg('rectangle', [abut['bw_width'],abut['backw_ht']], [0,abut['stem_ht']+abut['cap']])[1],
      cg('triangle_1', [abut['haunch']['stem_haunch'][0],-abut['haunch']['stem_haunch'][1]], [0,abut['stem_ht']])[1],
      cg('rectangle', [abut['haunch']['approach_slab_haunch_rec'][0],-abut['haunch']['approach_slab_haunch_rec'][1]], [0,abut['total_ht']-abut['approach_slab_dep']])[1],
      cg('triangle_1', [abut['haunch']['approach_slab_haunch_tri'][0],-abut['haunch']['approach_slab_haunch_tri'][1]], [0,abut['total_ht']-abut['approach_slab_dep']-abut['haunch']['approach_slab_haunch_rec'][1]])[1]]
mmt = []
for index, i in enumerate(H_force):
    mmt.append(round(H_force[index]*ht[index],5))
A7 = pd.DataFrame({"Weight":loads, 'Hz force(KN)':H_force,"Lever arm":ht,"Moment":mmt},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
# display(A7)
A7.loc[len(A7.index)] = [None, A7['Hz force(KN)'].sum(),None,A1['Moment'].sum()]
display(A7.rename(index={7:'Total'}))

,Weight,Hz force(KN),Lever arm,Moment
Stem(rectangle),964.6875,173.64375,1.83750,319.070390
Stem(triangle),38.5875,6.94575,1.22500,8.508540
Cap,157.5000,28.35000,3.92500,111.273750
Backwall,225.2250,40.54050,5.96250,241.722730
Stem haunch,10.5000,1.89000,3.50833,6.630740
Approach slab haunch(rec),13.1250,2.36250,6.57500,15.533440
Approach slab haunch(tri),6.5625,1.18125,6.28333,7.422180
Total,NaN,254.91375,NaN,-470.969121


Due to Earth pressure

In [17]:
C = ca(alpha, beta, delta, phi, lam, av)
k = ka(alpha, beta, delta, phi)
ep = (C-k)*h*unitwt_bfill 
A8 = pd.DataFrame({"Force":.5*ep*h*abut['width_of_abut'],"Lever arm":h/3,"Moment":(ep*h*abut['width_of_abut'])*h/3},index = ['Seismic force due to backfill'])
display(A8)

,Force,Lever arm,Moment
Seismic force due to backfill,754.755322,2.233333,3371.240436


Dynamic Surcharge 

In [18]:
A9 = pd.DataFrame({"Force":(unitwt_bfill*(C-k)*hl*abut['width_of_abut']*h),"Lever arm":(0.66*abut['total_ht']),"Moment":((unitwt_bfill*(C-k)*hl*abut['width_of_abut']*h)*(0.66*abut['total_ht']))},index = ['Dynamic Surcharge'])
display(A9)

,Force,Lever arm,Moment
Dynamic Surcharge,270.360115,4.62,1249.063732
